# Systemy uczące się - Zad. dom. 1: Minimalizacja ryzyka empirycznego
Celem zadania jest zaimplementowanie własnego drzewa decyzyjnego wykorzystującego idee minimalizacji ryzyka empirycznego.

### Autor rozwiązania
Uzupełnij poniższe informacje umieszczając swoje imię i nazwisko oraz numer indeksu:

In [ ]:
NAME = "Wojciech Kot"
ID = "151879"

## Twoja implementacja

Twoim celem jest uzupełnić poniższą klasę `TreeNode` tak by po wywołaniu `TreeNode.fit` tworzone było drzewo decyzyjne minimalizujące ryzyko empiryczne. Drzewo powinno wspierać problem klasyfikacji wieloklasowej (jak w przykładzie poniżej). Zaimplementowany algorytm nie musi (ale może) być analogiczny do zaprezentowanego na zajęciach algorytmu dla klasyfikacji. Wszelkie przejawy inwencji twórczej wskazane. **Pozostaw komenatrze w kodzie, które wyjaśniają Twoje rozwiązanie.**

Schemat oceniania:
- wynik na zbiorze Iris (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 10%: +40%,
- wynik na ukrytym zbiorze testowym 1 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 15%: +30%,
- wynik na ukrytym zbiorze testowym 2 (automatyczna ewaluacja) celność klasyfikacji >= prostego baseline'u + 5%: +30%.

Niedozwolone jest korzystanie z zewnętrznych bibliotek do tworzenia drzewa decyzyjnego (np. scikit-learn).
Możesz jedynie korzystać z biblioteki numpy.

#### Uwaga: Możesz dowolnie modyfikować elementy tego notebooka (wstawiać komórki i zmieniać kod), o ile będzie się w nim na koniec znajdowała kompletna implementacja klasy `TreeNode` w jednej komórce.

In [48]:
import numpy as np

class TreeNode:
	def __init__(self, min_samples_split=30, alpha=0.05):
		self.left: TreeNode | None = None  # wierzchołek znajdujący się po lewej stornie
		self.right: TreeNode | None = None  # wierzchołek znajdujący się po prawej stornie
		self.value = None  # wartość liścia i próg na węźle niebędącym liście
		self.arg = None # argument po którym następuje podział na danym węźle
		# zmienne odpowiadające za prunning, aby zapobiec przeuczeniu drzewa
		self.min_samples_split = min_samples_split
		self.alpha = alpha

	def entropy(self, y: np.ndarray) -> float:
		# Oblicza entropię zbioru docelowego y.
		unique, counts = np.unique(y, return_counts=True)
		p = counts / len(y)
		return -np.sum(p * np.log2(p))

	def best_split(self, data: np.ndarray, target: np.ndarray):
		# Oblicza najlepszy podział, na podstawie entropii
		n_samples, n_features = data.shape
		# zmienne best_ -> odpowiadają za finalnie zwracane wyniki, więc będą nadpisywane gdy znaleziony zostanie lepszy podział
		best_gain = 0
		best_split_value = None
		best_split_feature = None
		best_left_idx, best_right_idx = None, None # Indeksy do podziału na lewe i prawe poddrzewo

		base_entropy = self.entropy(target) # Obliczenie Entropii przed podziałem

		for feature in range(n_features):
				thresholds = np.unique(data[:, feature]) # potencjalne progi podziału (unikalne wartości)
				for threshold in thresholds:  		# iteracja po znalezionych progach
						# indeksy wartości większych i niewiększych od progu
						left_idx = data[:, feature] <= threshold
						right_idx = ~left_idx

						# warunek pominięcia podziału, dla zmiennem min_samples_split, aby nie przeuczać
						if np.sum(left_idx) < self.min_samples_split or np.sum(right_idx) < self.min_samples_split:
								continue

						# obliczanie 'zysku' entropii, czyli różnicy pomiędzy tym co było przedpodziałem a średniej ważonej entropii poddrzew po podziale
						left_entropy = self.entropy(target[left_idx])
						right_entropy = self.entropy(target[right_idx])
						weighted_entropy = (
								(np.sum(left_idx) / n_samples) * left_entropy +
								(np.sum(right_idx) / n_samples) * right_entropy
						)
						gain = base_entropy - weighted_entropy

						# no i jeśli znaleziono lepszy gain, to trzeba podpisać nowe zmienne, aby znaleźć najlepszy podział
						if gain > best_gain:
								best_gain = gain
								best_split_value = threshold
								best_split_feature = feature
								best_left_idx, best_right_idx = left_idx, right_idx

		return best_split_feature, best_split_value, best_left_idx, best_right_idx, best_gain

	def fit(self, data: np.ndarray, target: np.ndarray) -> None:
		"""
		Args:
			data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech
			target (np.ndarray): wektor klas o długości n, gdzie n to liczba przykładów
			Buduje drzewo na podstawie danych uczących
		"""
		feature, value, left_idx, right_idx, gain = self.best_split(data, target)
		if gain > self.alpha:
				# przypisanie po czym i na jakiej wartości nastąpił podział
				self.arg = feature
				self.value = value
				# budowa Lewego i prawego poddrzewa rekurencyjnie
				self.left = TreeNode()
				self.right = TreeNode()
				self.left.fit(data[left_idx], target[left_idx])
				self.right.fit(data[right_idx], target[right_idx])
		else: #no a jeśli nie ma poprawy po splicie, to liść.
				self.value = np.bincount(target).argmax()  # Przypisujemy najczęstszą klasę do liścia


	def pred_rec(self, data: np.ndarray):
		"""
		Args:
			data (np.ndarray): wektor cech danego przykładu
		"""
		if self.left is None and self.right is None:
			return self.value # if liść -> return
		else:
			#else sprawdź czy w prawo czy w lewo i tam idź
			if data[self.arg] <= self.value:
				return self.left.pred_rec(data)
			else:
				return self.right.pred_rec(data)

	def predict(self, data: np.ndarray) -> np.ndarray:
		"""
		Args:
			data (np.ndarray): macierz cech o wymiarach (n, m), gdzie n to liczba przykładów, a m to liczba cech

		Returns:
			np.ndarray: wektor przewidzoanych klas o długości n, gdzie n to liczba przykładów
		"""
		y_pred = np.zeros(data.shape[0])
		# dla każdego wektora danych uruchom pred_rec, które znajdzie dla niego klasę decyzyjną
		for i in range(data.shape[0]):
				y_pred[i] = self.pred_rec(data[i])
		return y_pred


## Przykład trenowanie i testowania drzewa

Później znajduje się przykład trenowania i testowania drzewa na zbiorze danych `iris`, który zawierający 150 próbek irysów, z czego każda próbka zawiera 4 atrybuty: długość i szerokość płatków oraz długość i szerokość działki kielicha. Każda próbka należy do jednej z trzech klas: `setosa`, `versicolor` lub `virginica`, które są zakodowane jak int.

Możesz go wykorzystać do testowania swojej implementacji. Możesz też zaimplementować własne testy lub użyć innych zbiorów danych, np. innych [zbiorów danych z scikit-learn](https://scikit-learn.org/stable/datasets/toy_dataset.html#toy-datasets).

In [71]:
from sklearn.datasets import load_iris, load_wine, load_digits, load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

"""
best for now: 30, 0.05 z wynikami:
0.88
0.8813559322033898
0.7508417508417509
0.9042553191489362
przy czym alpha miała w większości *znikomy* wpływ na wynik
"""
# te parametry można sobie tuningować, ale zakładam że jak zgłaszam zadanie, no to potrzebuję mieć te wartości już zadane...
mss = 30
a = 0.05

data = load_iris()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode(min_samples_split=mss, alpha=a)
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))


# a tu jest ten kod co był, skopiowany 4 razy z innymi datasetami, bo robienie pętli trwałoby dłużej, a student z natury jest leniwy
data = load_wine()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode(min_samples_split=mss, alpha=a)
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))


data = load_digits()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode(min_samples_split=mss, alpha=a)
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

data = load_breast_cancer()
X_train, X_test, y_train, y_test = train_test_split(data.data, data.target, test_size=0.33, random_state=2024)

tree_model = TreeNode(min_samples_split=mss, alpha=a)
tree_model.fit(X_train, y_train)
y_pred = tree_model.predict(X_test)
print(accuracy_score(y_test, y_pred))

0.88
0.8813559322033898
0.7508417508417509
0.9042553191489362
